<a href="https://colab.research.google.com/github/ramalahamir/image_classification_using_CNN/blob/main/Natural_scenes_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing datset from Kaggle


In [24]:
! pip install kaggle

In [25]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ramalahamir","key":"4bd49f59fcb670513208fcb32748b297"}'}

In [26]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [27]:
!kaggle datasets download -d puneet6060/intel-image-classification

Dataset URL: https://www.kaggle.com/datasets/puneet6060/intel-image-classification
License(s): copyright-authors
intel-image-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [28]:
!unzip intel-image-classification.zip -d intel-image-classification

Archive:  intel-image-classification.zip
replace intel-image-classification/seg_pred/seg_pred/10004.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Making our own mini CNN

In [29]:
import torch
import torch.nn as nn
import torch.optim as op
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

## Prepping our datasets:

In [30]:
# making our image transformer
transformed = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [31]:
train_path = "/content/intel-image-classification/seg_train/seg_train"
test_path = "/content/intel-image-classification/seg_test/seg_test"

In [32]:
train_dataset = datasets.ImageFolder(root = train_path, transform = transformed)
test_dataset = datasets.ImageFolder(root = test_path, transform = transformed)

In [33]:
print(train_dataset.classes)

['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


In [34]:
print(train_dataset.class_to_idx)

{'buildings': 0, 'forest': 1, 'glacier': 2, 'mountain': 3, 'sea': 4, 'street': 5}


## checking if our data is balanced

In [35]:
from collections import Counter

In [36]:
print(Counter(train_dataset.targets))

Counter({3: 2512, 2: 2404, 5: 2382, 4: 2274, 1: 2271, 0: 2191})


In [37]:
print(Counter(test_dataset.targets))

Counter({2: 553, 3: 525, 4: 510, 5: 501, 1: 474, 0: 437})


data is balanced!

In [38]:
# making batches
training_batches = DataLoader(train_dataset, batch_size = 32, shuffle = True)
testing_batches = DataLoader(test_dataset, batch_size = 32)

## Defining our CNN:

In [39]:
class my_CNN(nn.Module):
   def __init__(self):
      super().__init__()

      self.my_model = nn.Sequential(
          # layer 1:
          nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = (3,3), padding = 1),
          nn.BatchNorm2d(32),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size = (2,2), stride = 2),
          # output feature map size: 64x64x32

          # layer 2:
          nn.Conv2d(32, 64, 3, padding = 1),
          nn.BatchNorm2d(64),
          nn.ReLU(),
          nn.MaxPool2d(2,2),
          # output feature map size: 32x32x64

          # flattening for the neural network
          nn.Flatten(),
          # neural network layers (making 3 layers)
          nn.Linear(in_features = 32*32*64, out_features = 128),
          nn.BatchNorm1d(128),
          nn.ReLU(),
          nn.Linear(128, 64),
          nn.BatchNorm1d(64),
          nn.ReLU(),
          nn.Linear(64, 6),    # since 6 classes

          nn.Dropout(0.3)
      )

   def forward(self, image_batch):
     return self.my_model(image_batch)

## Training the model

In [40]:
miniCnn = my_CNN()
miniCnn.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
miniCnn.to(miniCnn.device)
loss_func = nn.CrossEntropyLoss()
optimizer = op.Adam(miniCnn.parameters(), lr = 0.0001)

In [41]:
print("Using device:", miniCnn.device)


Using device: cuda


In [42]:
counter = 0
patience = 10
best_acc = 0

for epoch in range(50):

  miniCnn.train()
  total_loss = 0

  for images, labels in training_batches:
    images, labels = images.to(miniCnn.device), labels.to(miniCnn.device)

    optimizer.zero_grad()
    outputs = miniCnn(images)
    loss = loss_func(outputs, labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  # validating the model
  miniCnn.eval()
  correct = 0
  total = 0

  with torch.no_grad():
    for images, labels in testing_batches:
      images, labels = images.to(miniCnn.device), labels.to(miniCnn.device)

      outputs = miniCnn(images)   # getting a tensor of class scores
      _, predicted = torch.max(outputs.data, 1)   # finding the max along horizontal axis
      total += labels.size(0)  # getting the batch size
      correct += (predicted == labels).sum().item()

  accuracy = 100*correct/total
  print(f"Epoch: ", {epoch}, " loss: ", {total_loss}, " accuracy: ", {accuracy})

  if accuracy > best_acc:
    best_acc = accuracy
    counter = 0
    torch.save(miniCnn.state_dict(), "best_model.pth")
  else:
    counter += 1
    if counter >= patience:
      print("Early stopping triggered")
      break

Epoch:  {0}  loss:  {559.6490975618362}  accuracy:  {67.43333333333334}
Epoch:  {1}  loss:  {473.0345444083214}  accuracy:  {72.93333333333334}
Epoch:  {2}  loss:  {442.00768303871155}  accuracy:  {72.53333333333333}
Epoch:  {3}  loss:  {421.3404833674431}  accuracy:  {73.9}
Epoch:  {4}  loss:  {404.8260888457298}  accuracy:  {75.76666666666667}
Epoch:  {5}  loss:  {391.1471453011036}  accuracy:  {77.03333333333333}
Epoch:  {6}  loss:  {382.8636432290077}  accuracy:  {77.43333333333334}
Epoch:  {7}  loss:  {376.70397022366524}  accuracy:  {77.2}
Epoch:  {8}  loss:  {369.2402574121952}  accuracy:  {79.9}
Epoch:  {9}  loss:  {361.32510259747505}  accuracy:  {78.8}
Epoch:  {10}  loss:  {354.93319845199585}  accuracy:  {79.26666666666667}
Epoch:  {11}  loss:  {350.4075959920883}  accuracy:  {77.66666666666667}
Epoch:  {12}  loss:  {344.74321565032005}  accuracy:  {80.66666666666667}
Epoch:  {13}  loss:  {345.6474344730377}  accuracy:  {80.86666666666666}
Epoch:  {14}  loss:  {335.903048992

## Testing our best trained model




In [1]:
# making a new model according to saved best states
my_model = my_CNN().to(miniCnn.device)
my_model.load_state_dict(torch.load("best_model.pth"))
my_model.eval()

NameError: name 'my_CNN' is not defined